<a href="https://colab.research.google.com/github/maninog/langchain/blob/main/LangChain_Indexes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Indexes

## README
- author: [Masumi Morishige](https://twitter.com/masumi_creator)
- created_at: 2023-05-02
- updated_at: 2023-05-06

### 実行方法
1. OpenAIのAPIキーを発行
2. `os.environ["OPENAI_API_KEY"] = "..."`の`""`の中にご自身のAPIキーを代入
3. 「ランタイム > すべてのセルを実行」を実行

### 参考情報
- Zenn: [LangChain Indexesとは？【Document Loaders・Text Splitters・Vectorstores】](https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_indexes)
- YouTube: [LangChain Indexesとは？【Document Loaders / Text Splitters / Vectorstores / Retrievers】](https://youtu.be/7tPMqrAoFAQ)

### OpenAI APIの発行方法

[<img src="https://img.youtube.com/vi/frpsKLNW1q4/maxresdefault.jpg" width="600px">](https://youtu.be/frpsKLNW1q4)

[【エンジニア向け】OpenAIのAPI連携方法【環境構築 + GASによるGoogle Documentへの組み込み】](https://youtu.be/frpsKLNW1q4)

## 環境構築

In [ ]:
!pip install langchain openai

In [ ]:
import os

#TODO: APIキーの登録が必要
os.environ["OPENAI_API_KEY"] = "..."

## 1. Document Loadersの使い方

In [ ]:
!pip3 install pypdf tiktoken chromadb

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

loader = PyPDFLoader("https://blog.freelance-jp.org/wp-content/uploads/2023/03/FreelanceSurvey2023.pdf")
pages = loader.load_and_split()
print(pages[0])

chroma_index = Chroma.from_documents(pages, OpenAIEmbeddings())
docs = chroma_index.similarity_search("「フリーランスのリモートワークの実態」について教えて。", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

## 2. Text Splittersの使い方

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)
text_list = text_splitter.split_text(long_text)
print(text_list)
print(len(text_list))

document_list = text_splitter.create_documents([long_text])
print(document_list)
print(len(document_list))

In [ ]:
len("GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。\n\n自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。")

## 3. Vectorstoresの使い方

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader

long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))
with open("./long_text.txt", "w") as f:
    f.write(long_text)
    f.close()

loader = TextLoader('./long_text.txt')

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)

index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, # Default
    embedding=OpenAIEmbeddings(), # Default
    text_splitter=text_splitter,
).from_loaders([loader])

query = "Q1. インターネット上の何のデータを使って、学習しているの？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

answer_with_sources = index.query_with_sources(query)
print(answer_with_sources)

query = "Q2. GPT4は第何世代のモデル？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

answer_with_sources = index.query_with_sources(query)
print(answer_with_sources)


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)

texts = text_splitter.split_text(long_text)

docsearch = Chroma.from_texts(texts, OpenAIEmbeddings())

query = "Q1. インターネット上の何のデータを使って、学習しているの？"
print(f"\n\n{query}")
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

query = "Q2. GPT4は第何世代のモデルは？"
print(f"\n\n{query}")
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

## 4. Retrieversの使い方

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)

document_list = text_splitter.create_documents([long_text])
print(document_list)
print(len(document_list))

db = FAISS.from_documents(document_list, OpenAIEmbeddings())
retriever = db.as_retriever()

query = "Q1. インターネット上の何のデータを使って、学習しているの？"
print(f"\n\n{query}")
docs = retriever.get_relevant_documents(query)
print(docs[0].page_content)